## EPL VAR Analysis
### Part 1 - Web Scraper
#### References
- https://stackoverflow.com/questions/42333498/selenium-get-a-list-of-all-the-elements-between-two-h1-elements

In [1]:
# Import necessary dependencies
from bs4 import BeautifulSoup
import urllib
import re
import time
import pandas as pd
import json
from datetime import datetime, date, timedelta

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime as dt

In [2]:
VAR_PAGE_2020_2021 = 'https://www.espn.com.sg/football/english-premier-league/story/4182135/how-var-decisions-affected-every-premier-league-club-in-2020-21'
VAR_PAGE_2019_2020 = 'https://www.espn.com/soccer/english-premier-league/story/3929823/how-var-decisions-have-affected-every-premier-league-club'

# Set wait times
waittime = 30
sleeptime = 0.5

In [3]:
# Initiate web driver
try:
    driver.close() # Close any existing WebDrivers
except Exception:
    pass

# Set webdriver options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('ignore-certificate-errors')

# Initiate webdriver
driver = webdriver.Chrome(options=options) 

___
### Get Team Statistics: 2020-2021

In [4]:
# Get driver to retrieve homepage
driver.get(VAR_PAGE_2020_2021)

# Wait for page to load
driver.implicitly_wait(waittime)

# Team numbering
team_list_elems = driver.find_elements_by_xpath("//div[@class='article-body']/h2")
team_list = []
net_score_list = []

for team in team_list_elems:
    team_name = team.text.rsplit(' ',1)[0]
    net_score = team.text.rsplit(' ',1)[-1]
    team_list.append(team_name)
    net_score_list.append(net_score)
    
all_elems = driver.find_elements_by_xpath("//div[@class='article-body']/p")

# Get general statistics for each of the 20 teams
team_stats_elems = [elem.text for elem in all_elems if 'Overturns: ' in elem.text]
team_stats_list = []

for stats in team_stats_elems:
    if 'Overturns: ' in stats:
        team_stats_list.append(stats)

data_tuples = list(zip(team_list,net_score_list, team_stats_list))
team_stats_df_2021 = pd.DataFrame(data_tuples, columns=['team_name', 'net_score', 'stats_combined'])
team_stats_df_2021

,team_name,net_score,stats_combined
0,Burnley,+5,Overturns: 6\nLeading to goals for: 1\nDisallo...
1,Chelsea,+3,Overturns: 8\nRejected overturns: 1\nLeading t...
2,Everton,+3,Overturns: 6\nRejected overturns: 1\nLeading t...
3,Fulham,+3,Overturns: 14\nRejected overturns: 1\nLeading ...
4,Manchester City,+3,Overturns: 7\nLeading to goals for: 2\nDisallo...
5,Brighton & Hove Albion,+2,Overturns: 15\nRejected overturns: 1\nLeading ...
6,Aston Villa,+1,Overturns: 10\nRejected overturns: 1\nLeading ...
7,Crystal Palace,+1,Overturns: 10\nRejected overturns: 1\nLeading ...
8,Leeds,+1,Overturns: 9\nLeading to goals for: 1\nDisallo...
9,Leicester City,+1,Overturns: 13\nLeading to goals for: 4\nDisall...


In [5]:
stats_col_mapping = [('overturns_total','Overturns'),
                    ('overturns_rejected','Rejected overturns'),
                    ('leading_to_goals_for','Leading to goals for'),
                    ('leading_to_goals_against','Leading to goals against'), 
                    ('disallowed_goals_for','Disallowed goals for'),
                    ('disallowed_goals_against','Disallowed goals against'),
                    ('net_goal_score','Net goal score'),
                    ('subj_decisions_for','Subjective decisions for'),
                    ('subj_decisions_against','Subjective decisions against'),
                    ('net_subjective_score','Net subjective score'),
                    ('penalties_for','Penalties for / against'),
                    ('penalties_against','Penalties for / against'),
                    ]

In [6]:
# Create columns
stats_col_list = [mapping[0] for mapping in stats_col_mapping]

for col in stats_col_list:
    team_stats_df_2021[col] = 0
    
# Update columns based on stats combined information
for i in range(len(team_stats_df_2021)):
    stats_info = team_stats_df_2021.iloc[i, :]['stats_combined']
    for line in stats_info.split('\n'):
        key = line.split(': ')[0]
        value = line.split(': ')[1]
        for mapping in stats_col_mapping:
            if mapping[1] == key:
                team_stats_df_2021.loc[i, mapping[0]] = value
                
# Amend penalties_for and penalties_against columns
team_stats_df_2021['penalties_for'] = team_stats_df_2021['penalties_for'].apply(lambda x: x.split(' / ')[0])

# Amend penalties_for and penalties_against columns
team_stats_df_2021['penalties_against'] = team_stats_df_2021['penalties_against'].apply(lambda x: x.split(' / ')[1])

# Add year column
team_stats_df_2021['year'] = '2020-2021'

# Drop stats_combined column
team_stats_df_2021.drop(columns = ['stats_combined'], inplace = True)

team_stats_df_2021

,team_name,net_score,overturns_total,overturns_rejected,leading_to_goals_for,leading_to_goals_against,disallowed_goals_for,disallowed_goals_against,net_goal_score,subj_decisions_for,subj_decisions_against,net_subjective_score,penalties_for,penalties_against,year
0,Burnley,+5,6,0,1,0,0,1,0,3,0,+3,1,0,2020-2021
1,Chelsea,+3,8,1,2,0,1,0,+1,2,1,+1,1,0,2020-2021
2,Everton,+3,6,1,2,0,1,1,+2,3,0,+3,1,0,2020-2021
3,Fulham,+3,14,1,1,1,2,5,+3,5,5,0,2,1,2020-2021
4,Manchester City,+3,7,0,2,0,2,0,0,2,1,+1,1,0,2020-2021
5,Brighton & Hove Albion,+2,15,1,2,3,2,3,0,4,4,0,2,3,2020-2021
6,Aston Villa,+1,10,1,0,0,3,3,0,4,3,+1,0,0,2020-2021
7,Crystal Palace,+1,10,1,2,1,0,1,0,4,4,0,2,1,2020-2021
8,Leeds,+1,9,0,1,1,2,1,-1,3,2,+1,0,1,2020-2021
9,Leicester City,+1,13,0,4,2,2,1,+1,3,3,0,3,2,2020-2021


### Get Team Statistics: 2019-2020

In [19]:
# Direct driver to 2019-2020 page
driver.get(VAR_PAGE_2019_2020)

# Team numbering
team_list_elems = driver.find_elements_by_xpath("//div[@class='article-body']/video1/h2") + \
                    driver.find_elements_by_xpath("//div[@class='article-body']/video1/video2/h2")
                    
team_list = []
net_score_list = []

for team in team_list_elems:
    team_name = team.text.rsplit(' ',1)[0]
    net_score = team.text.rsplit(' ',1)[-1]
    team_list.append(team_name)
    net_score_list.append(net_score)
    
all_elems = driver.find_elements_by_xpath("//div[@class='article-body']/video1/p") + \
            driver.find_elements_by_xpath("//div[@class='article-body']/video1/video2/p")

# Get general statistics for each of the 20 teams
team_stats_elems = [elem.text for elem in all_elems if 'Overturns: ' in elem.text]
team_stats_list = []

for stats in team_stats_elems:
    if 'Overturns: ' in stats:
        team_stats_list.append(stats)
        
data_tuples = list(zip(team_list,net_score_list, team_stats_list))
team_stats_df_1920 = pd.DataFrame(data_tuples, columns=['team_name', 'net_score', 'stats_combined'])

# Create columns
stats_col_list = [mapping[0] for mapping in stats_col_mapping]

for col in stats_col_list:
    team_stats_df_1920[col] = 0
    
# Update columns based on stats combined information
for i in range(len(team_stats_df_1920)):
    stats_info = team_stats_df_1920.iloc[i, :]['stats_combined']
    for line in stats_info.split('\n'):
        key = line.split(': ')[0]
        value = line.split(': ')[1]
        for mapping in stats_col_mapping:
            if mapping[1] == key:
                team_stats_df_1920.loc[i, mapping[0]] = value

2019/2020 data does not have penalties for/against information

In [20]:
# Amend penalties_for and penalties_against columns
team_stats_df_1920['penalties_for'] = ''
team_stats_df_1920['penalties_against'] = ''

# Add year column
team_stats_df_1920['year'] = '2019-2020'

# Drop unnecessary columns
team_stats_df_1920.drop(columns = ['stats_combined'], inplace = True)

team_stats_df_1920

,team_name,net_score,overturns_total,overturns_rejected,leading_to_goals_for,leading_to_goals_against,disallowed_goals_for,disallowed_goals_against,net_goal_score,subj_decisions_for,subj_decisions_against,net_subjective_score,penalties_for,penalties_against,year
0,Brighton & Hove Albion,+8,12,0,2,0,2,7,+7,2,0,+2,,,2019-2020
1,Manchester United,+7,13,0,1,2,0,7,+6,6,2,+4,,,2019-2020
2,Crystal Palace,+4,12,0,3,0,4,1,+2,6,2,+4,,,2019-2020
3,Burnley,+3,11,0,2,1,3,4,+2,4,2,+2,,,2019-2020
4,Newcastle,+3,3,0,1,0,0,0,+1,2,0,2,,,2019-2020
5,Southampton,+3,13,0,0,1,0,7,+6,1,4,-3,,,2019-2020
6,Liverpool,+2,8,0,1,0,3,4,+2,1,1,0,,,2019-2020
7,Leicester City,+1,15,0,1,1,3,4,+1,3,3,0,,,2019-2020
8,Tottenham Hotspur,+1,15,0,1,1,4,6,+2,3,3,0,,,2019-2020
9,Manchester City,0,16,0,3,2,4,2,-1,4,4,0,,,2019-2020


### Merge entire team statistics data

In [22]:
team_stats_df = team_stats_df_1920.append(team_stats_df_2021)

# For net score related columns, remove + sign
for col in team_stats_df.filter(like='net_').columns:
    team_stats_df[col] = team_stats_df[col].str.replace('+','', regex=True)
    
team_stats_df = team_stats_df.reset_index(drop = True)
team_stats_df

,team_name,net_score,overturns_total,overturns_rejected,leading_to_goals_for,leading_to_goals_against,disallowed_goals_for,disallowed_goals_against,net_goal_score,subj_decisions_for,subj_decisions_against,net_subjective_score,penalties_for,penalties_against,year
0,Brighton & Hove Albion,8,12,0,2,0,2,7,7,2,0,2,,,2019-2020
1,Manchester United,7,13,0,1,2,0,7,6,6,2,4,,,2019-2020
2,Crystal Palace,4,12,0,3,0,4,1,2,6,2,4,,,2019-2020
3,Burnley,3,11,0,2,1,3,4,2,4,2,2,,,2019-2020
4,Newcastle,3,3,0,1,0,0,0,1,2,0,2,,,2019-2020
5,Southampton,3,13,0,0,1,0,7,6,1,4,-3,,,2019-2020
6,Liverpool,2,8,0,1,0,3,4,2,1,1,0,,,2019-2020
7,Leicester City,1,15,0,1,1,3,4,1,3,3,0,,,2019-2020
8,Tottenham Hotspur,1,15,0,1,1,4,6,2,3,3,0,,,2019-2020
9,Manchester City,0,16,0,3,2,4,2,-1,4,4,0,,,2019-2020


In [23]:
# Export as CSV file
export_date = dt.today().strftime('%Y%m%d')
team_stats_df.to_csv(f'./data/EPL_VAR_Team_Stats_{export_date}.csv', index = False)

___
### Get incident statistics: 2020 - 2021

In [7]:
# Direct driver to 2020-2021 page
driver.get(VAR_PAGE_2020_2021)

# Team numbering
team_list_elems = driver.find_elements_by_xpath("//div[@class='article-body']/h2")
team_title_list = []
team_list = []

for team in team_list_elems:
    team_title_list.append(team.text)
    team_list.append(team.text.rsplit(' ',1)[0])

incident_list = []
inc_list = team_title_list + ['Game', 'Incident']
exc_list = ['Overturns:']

parent = driver.find_element_by_xpath("//div[@class='article-body']")
for child in parent.find_elements_by_xpath('./child::*'):
    if any(keyword in child.text for keyword in inc_list) and not any(keyword in child.text for keyword in exc_list):
        incident_list.append(child.text)

incident_list[1:]

['Burnley +5',
 'Game: Man City (A; Nov. 28)\nIncident: Bailey Peacock-Farrell own goal disallowed for offside in the build-up against Gabriel Jesus, 77th minute - FOR',
 'Game: Arsenal (A; Dec. 13)\nIncident: Granit Xhaka sent off for violent conduct on Ashley Westwood, 58th minute - FOR',
 'Game: Man United (H; Jan. 12)\nIncident: Man United free-kick and Robbie Brady red-card review cancelled for foul in the build-up by Luke Shaw, who was booked - NEUTRAL',
 'Game: West Brom (H; Feb. 20)\nIncident: Semi Ajayi sent off for handball and denying an obvious goal-scoring opportunity, 28th minute - FOR',
 'Game: Arsenal (H; March 6)\nIncident: Penalty and red card for handball against Erik Pieters cancelled, 85th minute - FOR',
 'Game: Southampton (A; April 4)\nIncident: Penalty awarded (scored by Chris Wood) for foul on Erik Pieters by Kyle Walker-Peters, 8th minute - FOR',
 'Chelsea +3',
 'Game: Liverpool (H; Sept. 20)\nIncident: Andreas Christensen sent off for denying a goal-scoring o

In [14]:
team_name = ''
incident_list_organized = []

for line in incident_list[1:]:
    global team_name
    
    for name in team_list:
        if line.startswith(name):
            team_name = name
            continue
    
    if line.startswith('Game'):
        incident_list_organized.append([team_name, line])
        
incident_list_organized

[['Burnley',
  'Game: Man City (A; Nov. 28)\nIncident: Bailey Peacock-Farrell own goal disallowed for offside in the build-up against Gabriel Jesus, 77th minute - FOR'],
 ['Burnley',
  'Game: Arsenal (A; Dec. 13)\nIncident: Granit Xhaka sent off for violent conduct on Ashley Westwood, 58th minute - FOR'],
 ['Burnley',
  'Game: Man United (H; Jan. 12)\nIncident: Man United free-kick and Robbie Brady red-card review cancelled for foul in the build-up by Luke Shaw, who was booked - NEUTRAL'],
 ['Burnley',
  'Game: West Brom (H; Feb. 20)\nIncident: Semi Ajayi sent off for handball and denying an obvious goal-scoring opportunity, 28th minute - FOR'],
 ['Burnley',
  'Game: Arsenal (H; March 6)\nIncident: Penalty and red card for handball against Erik Pieters cancelled, 85th minute - FOR'],
 ['Burnley',
  'Game: Southampton (A; April 4)\nIncident: Penalty awarded (scored by Chris Wood) for foul on Erik Pieters by Kyle Walker-Peters, 8th minute - FOR'],
 ['Chelsea',
  'Game: Liverpool (H; Sept

In [19]:
incident_df = pd.DataFrame.from_records(incident_list_organized, columns = ['team', 'match_info'])
incident_df

,team,match_info
0,Burnley,Game: Man City (A; Nov. 28)\nIncident: Bailey ...
1,Burnley,Game: Arsenal (A; Dec. 13)\nIncident: Granit X...
2,Burnley,Game: Man United (H; Jan. 12)\nIncident: Man U...
3,Burnley,Game: West Brom (H; Feb. 20)\nIncident: Semi A...
4,Burnley,Game: Arsenal (H; March 6)\nIncident: Penalty ...
...,...,...
176,West Brom,Game: Man City (H; Jan. 26)\nIncident: Ilkay G...
177,West Brom,Game: Man United (H; Feb. 14)\nIncident: Penal...
178,West Brom,Game: Burnley (A; Feb. 20)\nIncident: Semi Aja...
179,West Brom,Game: Brighton (H; Feb. 27)\nIncident: Penalty...


In [26]:
test_df = incident_df.copy()
test_df['test'] = incident_df.match_info.str.split("\n")
test_df

,team,match_info,test
0,Burnley,Game: Man City (A; Nov. 28)\nIncident: Bailey ...,"[Game: Man City (A; Nov. 28), Incident: Bailey..."
1,Burnley,Game: Arsenal (A; Dec. 13)\nIncident: Granit X...,"[Game: Arsenal (A; Dec. 13), Incident: Granit ..."
2,Burnley,Game: Man United (H; Jan. 12)\nIncident: Man U...,"[Game: Man United (H; Jan. 12), Incident: Man ..."
3,Burnley,Game: West Brom (H; Feb. 20)\nIncident: Semi A...,"[Game: West Brom (H; Feb. 20), Incident: Semi ..."
4,Burnley,Game: Arsenal (H; March 6)\nIncident: Penalty ...,"[Game: Arsenal (H; March 6), Incident: Penalty..."
...,...,...,...
176,West Brom,Game: Man City (H; Jan. 26)\nIncident: Ilkay G...,"[Game: Man City (H; Jan. 26), Incident: Ilkay ..."
177,West Brom,Game: Man United (H; Feb. 14)\nIncident: Penal...,"[Game: Man United (H; Feb. 14), Incident: Pena..."
178,West Brom,Game: Burnley (A; Feb. 20)\nIncident: Semi Aja...,"[Game: Burnley (A; Feb. 20), Incident: Semi Aj..."
179,West Brom,Game: Brighton (H; Feb. 27)\nIncident: Penalty...,"[Game: Brighton (H; Feb. 27), Incident: Penalt..."
